### Reading athletes file from bronze 

In [ ]:
from pyspark.sql.functions import regexp_replace,col
from pyspark.sql.types import *

In [ ]:
df=spark.read.parquet("abfss://bronze@olympicpro.dfs.core.windows.net/athletes")

In [ ]:
df.display()

### Transformations

In [ ]:
df=df.fillna({"birth_place":"Unknown","birth_country":"Unknown"})

In [ ]:
df=df.sort('height','weight',ascending=[False,True]).filter(col('weight')> 0)

In [ ]:
df=df.withColumn('nationality',regexp_replace('nationality','United States','USA'))

In [ ]:
df_grouped=df.groupBy('code').agg(count('code').alias('count')).filter(col('count')>1)

In [ ]:
df_renamed=df.withColumnRenamed('code','athlete_id')
df_renamed.display()

In [ ]:
df_list=df_renamed.withColumn('Occupation',split(col('occupation'),','))

In [ ]:
df_final=df_list.select('athlete_id',
 'current',
 'name',
 'name_short',
 'name_tv',
 'gender',
 'function',
 'country_code',
 'country',
 'country_long',
 'nationality',
 'nationality_long',
 'nationality_code',
 'height',
 'weight',
 'birth_date',
 'birth_place',
 'birth_country',)

### Writing it to Silver layer

In [ ]:
df_final.write.mode('overwrite')\
    .option("path","abfss://silver@olympicpro.dfs.core.windows.net/athletes") \
    .saveAsTable('olympiccatalog.silver_schema.athletes')